## Name: Aqsa Rahman
## Roll no: i191908
## Section DS-N

## Importing Libraries

In [76]:
import pandas as pd 
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F
from torchvision.io import read_image

## Reading files and preprocessing

In [77]:
labels= pd.read_csv("labels.csv")

In [78]:
labels.head()
#dropping the NaN values from our dataset.
labels=labels.dropna()

In [79]:
reference=pd.read_csv("reference.csv")
#dropping the NaN values from our dataset.
reference=reference.dropna()

## Bert Model-(for Embeddings)

In [80]:
batch_size, max_len = 512, 64
train_iter, vocab = d2l.load_data_wiki(batch_size, max_len)

In [81]:
net = d2l.BERTModel(len(vocab), num_hiddens=128, norm_shape=[128],
                    ffn_num_input=128, ffn_num_hiddens=256, num_heads=2,
                    num_layers=2, dropout=0.2, key_size=128, query_size=128,
                    value_size=128, hid_in_features=128, mlm_in_features=128,
                    nsp_in_features=128)
devices = d2l.try_all_gpus()
loss = nn.CrossEntropyLoss()

In [82]:
def _get_batch_loss_bert(net, loss, vocab_size, tokens_X,
                         segments_X, valid_lens_x,
                         pred_positions_X, mlm_weights_X,
                         mlm_Y, nsp_y):
    # Forward pass
    _, mlm_Y_hat, nsp_Y_hat = net(tokens_X, segments_X,
                                  valid_lens_x.reshape(-1),
                                  pred_positions_X)
    # Compute masked language model loss
    mlm_l = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y.reshape(-1)) *\
    mlm_weights_X.reshape(-1, 1)
    mlm_l = mlm_l.sum() / (mlm_weights_X.sum() + 1e-8)
    # Compute next sentence prediction loss
    nsp_l = loss(nsp_Y_hat, nsp_y)
    l = mlm_l + nsp_l
    return mlm_l, nsp_l, l

In [83]:
def train_bert(train_iter, net, loss, vocab_size, devices, num_steps):
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    trainer = torch.optim.Adam(net.parameters(), lr=0.01)
    step, timer = 0, d2l.Timer()
    animator = d2l.Animator(xlabel='step', ylabel='loss',
                            xlim=[1, num_steps], legend=['mlm', 'nsp'])
    # Sum of masked language modeling losses, sum of next sentence prediction
    # losses, no. of sentence pairs, count
    metric = d2l.Accumulator(4)
    num_steps_reached = False
    while step < num_steps and not num_steps_reached:
        for tokens_X, segments_X, valid_lens_x, pred_positions_X,\
            mlm_weights_X, mlm_Y, nsp_y in train_iter:
            tokens_X = tokens_X.to(devices[0])
            segments_X = segments_X.to(devices[0])
            valid_lens_x = valid_lens_x.to(devices[0])
            pred_positions_X = pred_positions_X.to(devices[0])
            mlm_weights_X = mlm_weights_X.to(devices[0])
            mlm_Y, nsp_y = mlm_Y.to(devices[0]), nsp_y.to(devices[0])
            trainer.zero_grad()
            timer.start()
            mlm_l, nsp_l, l = _get_batch_loss_bert(
                net, loss, vocab_size, tokens_X, segments_X, valid_lens_x,
                pred_positions_X, mlm_weights_X, mlm_Y, nsp_y)
            l.backward()
            trainer.step()
            metric.add(mlm_l, nsp_l, tokens_X.shape[0], 1)
            timer.stop()
            animator.add(step + 1,
                         (metric[0] / metric[3], metric[1] / metric[3]))
            step += 1
            if step == num_steps:
                num_steps_reached = True
                break

    print(f'MLM loss {metric[0] / metric[3]:.3f}, '
          f'NSP loss {metric[1] / metric[3]:.3f}')
    print(f'{metric[2] / timer.sum():.1f} sentence pairs/sec on '
          f'{str(devices)}')

In [84]:
#train_bert(train_iter, net, loss, len(vocab), devices, 50)

In [85]:
def get_bert_encoding(net, tokens_a, tokens_b=None):
    tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments = torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoded_X

In [86]:
tokens_a = ['a', 'crane', 'is', 'flying']
encoded_text = get_bert_encoding(net, tokens_a)
# Tokens: '<cls>', 'a', 'crane', 'is', 'flying', '<sep>'
encoded_text_cls = encoded_text[:, 0, :]
encoded_text_crane = encoded_text[:, 2, :]
encoded_text.shape, encoded_text_cls.shape, encoded_text_crane[0][:3]

(torch.Size([1, 6, 128]),
 torch.Size([1, 128]),
 tensor([ 0.2624, -0.2890,  1.0206], grad_fn=<SliceBackward0>))

## Image processing


In [12]:
from PIL import Image
import os, sys

In [ ]:
#adding image names to a list from the dataframe
image_new= labels['image_name']
image_new=image_new.tolist()

In [ ]:
for i in image_new:
    image = Image.open("images/"+i)
    #resizing the image to 600x600
    image = image.resize((120,120),Image.ANTIALIAS)
    #converting to RGB AND PNG(image-type)
    image = image.convert('RGB')
    new_name = i.split('.')[0]+".png"
    #saving the processed images in a new file
    image.save(fp="new_images/"+new_name)

## Splitting into Train, Test, and Validate

####  label encoding

In [87]:
#label encoding the image name data to provide specific labels to different classes.
def label_encoding(labels):
    labels = labels[['image_name','text_corrected','overall_sentiment']]
    labels['image_name'] = [i.split(".")[0] for i in labels['image_name'] ]
    labels['overall_sentiment'] = labels['overall_sentiment'].replace("very_positive", 1)
    labels['overall_sentiment'] = labels['overall_sentiment'].replace("positive", 1)
    labels['overall_sentiment'] = labels['overall_sentiment'].replace("very_negative", 2)
    labels['overall_sentiment'] = labels['overall_sentiment'].replace("negative", 2)
    labels['overall_sentiment'] = labels['overall_sentiment'].replace("neutral", 0)
    
    return labels

In [88]:
labels=label_encoding(labels)

<ipython-input-87-8369cb02509a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['image_name'] = [i.split(".")[0] for i in labels['image_name'] ]
<ipython-input-87-8369cb02509a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['overall_sentiment'] = labels['overall_sentiment'].replace("very_positive", 1)
<ipython-input-87-8369cb02509a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [89]:
#train=60,test=20,validate=20
train, validate, test = np.split(labels.sample(frac=1, random_state=30), [int(.6*len(labels)), int(.8*len(labels))])

In [90]:
#Saving into new csv files.
train.to_csv("train.csv",index=False)
validate.to_csv("validate.csv",index=False)
test.to_csv("test.csv",index=False)

## Data loader for neural network

In [102]:
""" Neural network (ANN) for images data to classify in 5 labels"""
from torch.utils.data import Dataset
class Dataset_loader(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        train_labels = pd.read_csv(annotations_file)
        labels_images = [i for i in train_labels['overall_sentiment']]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]+'.png')
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        text = self.img_labels.iloc[idx, 1]
        text = text.split()
        padding = 187 - len(text)
        if(padding>0):
            for i in range(padding):
                text.append('')
        else:
            text = text[0:187]
        encoded_text = get_bert_encoding(net,text)
        array = torch.as_tensor(label)
        x=image.float()
        return x,encoded_text,array

In [103]:
def loader():
    train = Dataset_loader(annotations_file="train.csv",img_dir="./new_images")
    test = Dataset_loader(annotations_file="test.csv",img_dir="./new_images")
    validate = Dataset_loader(annotations_file="validate.csv",img_dir="./new_images")
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=64,num_workers=0,pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=64)
    validate_dataloader = torch.utils.data.DataLoader(validate, batch_size=64)
    return train_dataloader,test_dataloader,validate_dataloader 


In [104]:
train_dataloader,test_dataloader,validate_dataloader = loader()

In [105]:
test = Dataset_loader(annotations_file="test.csv",img_dir="./new_images")
test.__getitem__(0)

(tensor([[[110., 105., 114.,  ...,  56.,  57.,  59.],
          [103., 103., 103.,  ...,  57.,  59.,  59.],
          [112.,  96.,  64.,  ...,  69.,  62.,  60.],
          ...,
          [ 37.,  38.,  43.,  ..., 102., 170., 197.],
          [ 56.,  57.,  55.,  ..., 181., 197., 213.],
          [ 58.,  59.,  60.,  ..., 224., 224., 225.]],
 
         [[ 61.,  61.,  76.,  ...,  39.,  40.,  43.],
          [ 62.,  69.,  66.,  ...,  43.,  42.,  41.],
          [ 68.,  70.,  48.,  ...,  61.,  53.,  53.],
          ...,
          [ 40.,  44.,  53.,  ..., 104., 170., 198.],
          [ 57.,  62.,  62.,  ..., 179., 195., 213.],
          [ 61.,  63.,  65.,  ..., 227., 224., 225.]],
 
         [[ 50.,  43.,  56.,  ...,  35.,  38.,  41.],
          [ 47.,  50.,  53.,  ...,  38.,  39.,  37.],
          [ 53.,  55.,  38.,  ...,  46.,  40.,  39.],
          ...,
          [ 37.,  41.,  54.,  ..., 112., 173., 197.],
          [ 60.,  64.,  68.,  ..., 183., 197., 213.],
          [ 63.,  63.,  68.,  .

## Image Neural Network 


In [118]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(43200,2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.fc4 = nn.Linear(500, 100)
        self.fc5 = nn.Linear(100, 3)
        
    
    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

        x =F.log_softmax(self.fc5(x))
        return x
image_net = Net()

In [119]:
Net()

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=43200, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=500, bias=True)
  (fc4): Linear(in_features=500, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=3, bias=True)
)

##  Text Neural Network

In [120]:
class TextNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(24192,5000)
        self.fc2 = nn.Linear(5000, 3000)
        self.fc3 = nn.Linear(3000, 1500)
        self.fc4 = nn.Linear(1500, 500)
        self.fc5 = nn.Linear(500, 100)
        self.fc6 = nn.Linear(100, 3)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))


        x =self.fc6(x)
        return x
nettext = TextNet()

## Combining two models

In [121]:
class Joining_models(nn.Module):
    def __init__(self, model1, model2):
        super(Joining_models, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.classifier = nn.Linear(6,3)
    def forward(self, x1, x2):
        x1 = self.model1(x1)
        x2 = self.model2(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(x)
        
        return x
    

In [122]:
finalmodel=Joining_models(image_net,nettext)

In [123]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(finalmodel.parameters(), lr=0.001)

## Loss function

In [128]:
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs,text, labels = data
        optimizer.zero_grad()
        outputs = finalmodel(inputs,text)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2 == 0:   
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f}')
            running_loss = 0.0

<ipython-input-118-a9cf0ed24942>:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x =F.log_softmax(self.fc5(x))


[1,     1] loss: 0.534
[1,     3] loss: 0.975
[1,     5] loss: 1.152
[1,     7] loss: 0.953
[1,     9] loss: 1.019
[1,    11] loss: 0.900
[1,    13] loss: 0.919
[1,    15] loss: 1.049
[1,    17] loss: 0.981
[1,    19] loss: 0.908
[1,    21] loss: 0.928
[1,    23] loss: 0.886
[1,    25] loss: 0.921
[1,    27] loss: 0.916
[1,    29] loss: 0.845
[1,    31] loss: 0.909
[1,    33] loss: 0.885
[1,    35] loss: 0.928
[1,    37] loss: 0.861
[1,    39] loss: 0.803
[1,    41] loss: 0.837
[1,    43] loss: 1.002
[1,    45] loss: 0.959
[1,    47] loss: 0.922
[1,    49] loss: 0.875
[1,    51] loss: 0.950
[1,    53] loss: 0.924
[1,    55] loss: 0.923
[1,    57] loss: 0.845
[1,    59] loss: 0.892
[1,    61] loss: 0.902
[1,    63] loss: 0.944
[1,    65] loss: 1.406
[2,     1] loss: 0.459
[2,     3] loss: 0.962
[2,     5] loss: 0.968
[2,     7] loss: 0.887
[2,     9] loss: 0.916
[2,    11] loss: 0.912
[2,    13] loss: 0.962
[2,    15] loss: 1.064
[2,    17] loss: 0.953
[2,    19] loss: 0.883
[2,    21] 

## Accuracy

In [129]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_dataloader:
        inputs,text,labels = data
        outputs = nettext(text)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy  = {100 * correct // total } percent')

Accuracy  = 58 percent


## Saving the model

In [131]:
PATH = './Ai_project.pth'
torch.save(net.state_dict(), PATH)